In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [4]:
df = pd.read_csv('cleaned_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3448 entries, 0 to 3447
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   president              3448 non-null   object
 1   question_order         3448 non-null   int64 
 2   interview_question     3448 non-null   object
 3   interview_answer       3448 non-null   object
 4   question               3448 non-null   object
 5   inaudible              3448 non-null   bool  
 6   multiple_questions     3448 non-null   bool  
 7   affirmative_questions  3448 non-null   bool  
 8   index                  3448 non-null   int64 
 9   clarity_label          3448 non-null   object
 10  evasion_label          3448 non-null   object
dtypes: bool(3), int64(2), object(6)
memory usage: 225.7+ KB


In [5]:
TARGET_COLUMN = 'evasion_label'
df['question'] = df['question'].astype(str).fillna('')
df['interview_answer'] = df['interview_answer'].astype(str).fillna('')
df[TARGET_COLUMN] = df[TARGET_COLUMN].astype(str)

print(f"Classes distribution {TARGET_COLUMN}:")
print(df[TARGET_COLUMN].value_counts())

Classes distribution evasion_label:
evasion_label
Explicit               1052
Dodging                 706
Implicit                488
General                 386
Deflection              381
Declining to answer     145
Claims ignorance        119
Clarification            92
Partial/half-answer      79
Name: count, dtype: int64


In [6]:
df['text_input'] = df['question'] + " [SEP] " + df['interview_answer']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text_input'],
    df[TARGET_COLUMN],
    test_size=0.2,
    random_state=42,
    stratify=df[TARGET_COLUMN]
)
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [8]:
model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train_vec, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

In [9]:
preds = model.predict(X_test_vec)

print(f"Accuracy: {accuracy_score(y_test, preds):.4f}")
print("\nDetailed report (Precision, Recall, F1):")
print(classification_report(y_test, preds))

cm = confusion_matrix(y_test, preds)
print("\nConfusion Matrix:")
print(cm)

Accuracy: 0.2681

Detailed report (Precision, Recall, F1):
                     precision    recall  f1-score   support

   Claims ignorance       0.22      0.38      0.28        24
      Clarification       0.29      0.78      0.42        18
Declining to answer       0.20      0.34      0.25        29
         Deflection       0.19      0.29      0.23        76
            Dodging       0.30      0.22      0.25       141
           Explicit       0.49      0.23      0.32       211
            General       0.27      0.29      0.28        77
           Implicit       0.23      0.26      0.24        98
Partial/half-answer       0.08      0.19      0.12        16

           accuracy                           0.27       690
          macro avg       0.25      0.33      0.26       690
       weighted avg       0.32      0.27      0.27       690


Confusion Matrix:
[[ 9  1  2  3  4  2  0  2  1]
 [ 0 14  0  0  3  0  1  0  0]
 [ 1  5 10  4  3  1  2  1  2]
 [ 6  2  3 22 13  3  8 18  1]
 [ 7  

In [10]:
from sklearn import svm

svm_clf = svm.SVC(kernel='linear',class_weight='balanced', random_state=42)
svm_clf.fit(X_train_vec, y_train)

,"C C: float, default=1.0Regularization parameter. The strength of the regularization isinversely proportional to C. Must be strictly positive. The penaltyis a squared l2 penalty. For an intuitive visualization of the effectsof scaling the regularization parameter C, see:ref:`sphx_glr_auto_examples_svm_plot_svm_scale_c.py`.",1.0
,"kernel kernel: {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'} or callable, default='rbf'Specifies the kernel type to be used in the algorithm. Ifnone is given, 'rbf' will be used. If a callable is given it is used topre-compute the kernel matrix from data matrices; that matrix should bean array of shape ``(n_samples, n_samples)``. For an intuitivevisualization of different kernel types see:ref:`sphx_glr_auto_examples_svm_plot_svm_kernels.py`.",'linear'
,"degree degree: int, default=3Degree of the polynomial kernel function ('poly').Must be non-negative. Ignored by all other kernels.",3
,"gamma gamma: {'scale', 'auto'} or float, default='scale'Kernel coefficient for 'rbf', 'poly' and 'sigmoid'.- if ``gamma='scale'`` (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,- if 'auto', uses 1 / n_features- if float, must be non-negative... versionchanged:: 0.22 The default value of ``gamma`` changed from 'auto' to 'scale'.",'scale'
,"coef0 coef0: float, default=0.0Independent term in kernel function.It is only significant in 'poly' and 'sigmoid'.",0.0
,"shrinking shrinking: bool, default=TrueWhether to use the shrinking heuristic.See the :ref:`User Guide `.",True
,"probability probability: bool, default=FalseWhether to enable probability estimates. This must be enabled priorto calling `fit`, will slow down that method as it internally uses5-fold cross-validation, and `predict_proba` may be inconsistent with`predict`. Read more in the :ref:`User Guide `.",False
,"tol tol: float, default=1e-3Tolerance for stopping criterion.",0.001
,"cache_size cache_size: float, default=200Specify the size of the kernel cache (in MB).",200
,"class_weight class_weight: dict or 'balanced', default=NoneSet the parameter C of class i to class_weight[i]*C forSVC. If not given, all classes are supposed to haveweight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.",'balanced'
,"verbose verbose: bool, default=FalseEnable verbose output. Note that this setting takes advantage of aper-process runtime setting in libsvm that, if enabled, may not workproperly in a multithreaded context.",False


In [11]:
preds = svm_clf.predict(X_test_vec)

print(f"Accuracy: {accuracy_score(y_test, preds):.4f}")
print("\nDetailed report (Precision, Recall, F1):")
print(classification_report(y_test, preds))

cm = confusion_matrix(y_test, preds)
print("\nConfusion Matrix:")
print(cm)

Accuracy: 0.2870

Detailed report (Precision, Recall, F1):
                     precision    recall  f1-score   support

   Claims ignorance       0.27      0.29      0.28        24
      Clarification       0.50      0.67      0.57        18
Declining to answer       0.30      0.31      0.31        29
         Deflection       0.19      0.34      0.24        76
            Dodging       0.29      0.26      0.27       141
           Explicit       0.52      0.25      0.34       211
            General       0.24      0.27      0.25        77
           Implicit       0.23      0.32      0.26        98
Partial/half-answer       0.13      0.19      0.15        16

           accuracy                           0.29       690
          macro avg       0.30      0.32      0.30       690
       weighted avg       0.34      0.29      0.29       690


Confusion Matrix:
[[ 7  1  1  4  6  2  0  2  1]
 [ 0 12  0  0  5  0  1  0  0]
 [ 0  0  9  4  7  3  2  2  2]
 [ 4  0  1 26 12  3 11 19  0]
 [ 8  